<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

## Automatic Test Generation

In this project, we will build an automatic test generation and grading platform!
All we have to do is to provide a topic, the number of questions and the number of options for each question!
Based on this information, a suitable test is generated, presented to the user and graded automatically!

## Imports

In [1]:
import os
from openai import OpenAI


## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.



In [2]:
client = OpenAI()

### Tell GPT how to generate the test

We tell GPT to create a multiple choiz quiz. Hence we define the topic, the number of possible answers as well as the number of questions.
To enable automatical grading later, GPT needs to incorporate the correct answer!


In [3]:
def create_system_prompt():
    system_prompt = "You are a helpful assistant for test generation."
    return system_prompt

In [4]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions. " \
                 + f"Each question should have {num_possible_answers} options. "\
                 + f"Also include the correct answer for each question using the starting string 'Correct Answer: '."
    return prompt

In [5]:
create_test_prompt("Python", 4, 4)

"Create a multiple choice quiz on the topic of Python consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer: '."

### OpenAI API Call
Let's use gpt-3.5-turbo

In [6]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": create_system_prompt()},
    {"role": "user", "content": create_test_prompt("Python", 4, 4)},
  ]
)


In [7]:
response

ChatCompletion(id='chatcmpl-8QVPZTsRX758MZKkiiW7RC7oSRWzN', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Here is a multiple choice quiz on the topic of Python:\n\n1. What is Python?\n   a) A high-level programming language\n   b) A type of snake\n   c) A mathematical equation\n   d) A computer hardware\n\nCorrect Answer: a) A high-level programming language\n\n2. Which of the following is NOT a data type in Python?\n   a) String\n   b) Integer\n   c) Float\n   d) Boolean\n\nCorrect Answer: d) Boolean\n\n3. What is the correct way to declare a variable in Python?\n   a) var x = 5\n   b) int x = 5\n   c) x = 5\n   d) declare x = 5\n\nCorrect Answer: c) x = 5\n\n4. What is the output of the following code?\n   x = 5\n   y = 2\n   print(x + y * 2)\n\n   a) 9\n   b) 10\n   c) 12\n   d) 7\n\nCorrect Answer: b) 10', role='assistant', function_call=None, tool_calls=None))], created=1701327965, model='gpt-3.5-turbo-0613', object='chat.completion', system

In [8]:
print(response.choices[0].message.content)

Here is a multiple choice quiz on the topic of Python:

1. What is Python?
   a) A high-level programming language
   b) A type of snake
   c) A mathematical equation
   d) A computer hardware

Correct Answer: a) A high-level programming language

2. Which of the following is NOT a data type in Python?
   a) String
   b) Integer
   c) Float
   d) Boolean

Correct Answer: d) Boolean

3. What is the correct way to declare a variable in Python?
   a) var x = 5
   b) int x = 5
   c) x = 5
   d) declare x = 5

Correct Answer: c) x = 5

4. What is the output of the following code?
   x = 5
   y = 2
   print(x + y * 2)

   a) 9
   b) 10
   c) 12
   d) 7

Correct Answer: b) 10


### Q/A Extraction

We now need to extract the questions and answers to present them to the students later

In [9]:
def create_student_view(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view
 

In [10]:
create_student_view(response.choices[0].message.content, 4)

{1: 'Here is a multiple choice quiz on the topic of Python:\n\n1. What is Python?\n   a) A high-level programming language\n   b) A type of snake\n   c) A mathematical equation\n   d) A computer hardware\n\n',
 2: '\n2. Which of the following is NOT a data type in Python?\n   a) String\n   b) Integer\n   c) Float\n   d) Boolean\n\n',
 3: '\n3. What is the correct way to declare a variable in Python?\n   a) var x = 5\n   b) int x = 5\n   c) x = 5\n   d) declare x = 5\n\n',
 4: '\n4. What is the output of the following code?\n   x = 5\n   y = 2\n   print(x + y * 2)\n\n   a) 9\n   b) 10\n   c) 12\n   d) 7\n\n'}

In [11]:
def extract_answers(test, num_questions):
    answers = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line+"\n"

            if question_number < num_questions:
                question_number+=1
                answers[question_number] = ""
    return answers



In [12]:
extract_answers(response.choices[0].message.content, 4)

{1: 'Correct Answer: a) A high-level programming language\n',
 2: 'Correct Answer: d) Boolean\n',
 3: 'Correct Answer: c) x = 5\n',
 4: 'Correct Answer: b) 10\n'}

### Exam simulation
Based on the extracted questions, we can now simulate the exam

In [13]:
def take(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers


In [14]:
student_answers = take(create_student_view(response.choices[0].message.content, 4))

Here is a multiple choice quiz on the topic of Python:

1. What is Python?
   a) A high-level programming language
   b) A type of snake
   c) A mathematical equation
   d) A computer hardware


Enter your answer: A

2. Which of the following is NOT a data type in Python?
   a) String
   b) Integer
   c) Float
   d) Boolean


Enter your answer: D

3. What is the correct way to declare a variable in Python?
   a) var x = 5
   b) int x = 5
   c) x = 5
   d) declare x = 5


Enter your answer: c

4. What is the output of the following code?
   x = 5
   y = 2
   print(x + y * 2)

   a) 9
   b) 10
   c) 12
   d) 7


Enter your answer: a


### Automatic Grading
Based on the student's answers and correct answers, we can now grade the test!

In [15]:
def grade(correct_answer_dict, answers):
    correct_answers = 0
    for question, answer in answers.items():
        if answer.upper() == correct_answer_dict[question].upper()[16]:
            correct_answers+=1
    grade = 100 * correct_answers / len(answers)

    if grade < 60:
        passed = "Not passed!"
    else:
        passed = "Passed!"
    return f"{correct_answers} out of {len(answers)} correct! You achieved: {grade} % : {passed}"


In [16]:
grade(extract_answers(response.choices[0].message.content, 4), student_answers)

'3 out of 4 correct! You achieved: 75.0 % : Passed!'